In [2]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
score_dir = "data/scores"

In [4]:
box_scores = os.listdir(score_dir)

In [7]:
box_scores = [os.path.join(score_dir, f) for f in box_scores if f.endswith(".html")]

In [13]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.head")]
    return soup

In [27]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {"id":"line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols

    line_score = line_score[["team", "total"]]
    return line_score

In [42]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {"id": f"box-{team}-game-{stat}"}, index_col = 0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [55]:
box_score = box_scores[0]
soup = parse_html(box_score)
line_score = read_line_score(soup)
teams = list(line_score["team"])

summaries = []
for team in teams:
    basic = read_stats(soup, team, "basic")
    advanced = read_stats(soup, team, "advanced")

    totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
    totals = totals.index.str.lower()
        
    maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
    maxes.index = maxes.index.str.lower() + "_max"

    summary = pd.concat([totals, maxes])

TypeError: cannot concatenate object of type '<class 'pandas.core.indexes.base.Index'>'; only Series and DataFrame objs are valid

In [54]:
summary


,ATL
mp,mp
fg,fg
fga,fga
fg%,fg%
3p,3p
...,...
tov%_max,57.1
usg%_max,33.8
ortg_max,258.0
drtg_max,121.0
